# RAG: Recuperación y Generación Aumentada con Gemini

## Construyendo un Sistema Inteligente con Documentos Empresariales

En este cuaderno aprenderemos a crear un sistema RAG (Retrieval Augmented Generation) que puede responder preguntas usando documentos de nuestra empresa como fuente de conocimiento.

**¿Qué es RAG?** Es como tener un asistente súper inteligente que:
1. **Busca** información relevante en nuestros documentos
2. **Genera** respuestas basadas en esa información específica

### ¿Por qué RAG es revolucionario?

Los modelos de IA como ChatGPT o Gemini tienen un "punto ciego": solo conocen información hasta su fecha de entrenamiento y no saben nada específico sobre nuestra empresa. RAG soluciona esto permitiendo que el modelo use nuestros documentos internos para dar respuestas precisas y actualizadas.

### Lo que construiremos hoy:
- Un sistema que puede leer documentos de texto de nuestra empresa
- Una base de conocimiento inteligente que entiende el contexto
- Un asistente que responde preguntas usando solo información verificada de nuestros documentos

## Configuración del Entorno

Primero instalamos las herramientas necesarias. LangChain es nuestra "caja de herramientas" principal para RAG.

In [ ]:
# Instalamos las librerías necesarias para nuestro sistema RAG
# LangChain: la librería más popular para construir sistemas RAG de forma simple
# ChromaDB: nuestra base de datos vectorial para guardar los documentos
# Google GenerativeAI: para conectar con Gemini (solo para generación final)
# sentence-transformers: para embeddings locales multilenguaje
!pip install langchain langchain-google-genai langchain-chroma chromadb sentence-transformers -q

print("Todas las librerías instaladas correctamente")
print("IMPORTANTE: Solo se usará API de Gemini para generación final de respuestas")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 7.6 MB/s eta 0:0

In [ ]:
# Importamos todas las herramientas que vamos a necesitar
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from chromadb.utils import embedding_functions

# Configuración para mostrar mejor los resultados
import warnings
warnings.filterwarnings('ignore')

print("Librerías cargadas exitosamente")
print("Usando embeddings locales para reducir uso de API")

Librerías cargadas exitosamente
Usando embeddings locales para reducir uso de API


## Configuración de la API de Gemini

Necesitamos configurar nuestra conexión con Gemini de Google. Este será el "cerebro" que generará las respuestas finales.

In [ ]:
# Detectamos si estamos en Google Colab o en un entorno local
try:
    import google.colab
    from google.colab import userdata
    IN_COLAB = True
    print("🔍 Entorno detectado: Google Colab")
except ImportError:
    IN_COLAB = False
    print("🔍 Entorno detectado: Local")

# Obtenemos la clave API según el entorno
if IN_COLAB:
    # En Colab: usar los secretos de Colab (más seguro)
    try:
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        print("✅ Clave API cargada desde secretos de Colab")
    except Exception as e:
        print("❌ No se encontró GOOGLE_API_KEY en los secretos de Colab")
        print("   Ve a la barra lateral izquierda > 🔑 Secretos > Agregar GOOGLE_API_KEY")
        GOOGLE_API_KEY = input("Pega tu clave API de Google aquí: ")
else:
    # En local: usar variable de entorno
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        print("❌ No se encontró GOOGLE_API_KEY en las variables de entorno")
        print("   Opción 1: Agrega GOOGLE_API_KEY a tu archivo .env")
        print("   Opción 2: Ejecuta: export GOOGLE_API_KEY=tu_clave_aqui")
        GOOGLE_API_KEY = input("Pega tu clave API de Google aquí: ")
    else:
        print("✅ Clave API cargada desde variables de entorno")

# Configuramos la variable de entorno para que LangChain la use
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
print("🚀 Configuración de Gemini completada")

🔍 Entorno detectado: Google Colab
✅ Clave API cargada desde secretos de Colab
🚀 Configuración de Gemini completada


## Creando Documentos de Prueba

Como esto es educativo, vamos a crear documentos de ejemplo que simulan información empresarial típica. En un caso real, cargarías tus propios documentos desde archivos PDF, Word, o bases de datos.

In [ ]:
# Creamos documentos de ejemplo que simulan información empresarial real
# En la práctica, estos datos vendrían de archivos PDF, Word, bases de datos, etc.

documentos_empresa = [
    {
        "titulo": "Política de Recursos Humanos 2024",
        "contenido": """
        Política de Vacaciones: Todos los empleados tienen derecho a 21 días hábiles de vacaciones anuales.
        Las vacaciones deben solicitarse con al menos 15 días de anticipación.
        Durante enero y febrero, solo el 30% del personal puede tomar vacaciones simultáneamente.

        Horarios de Trabajo: La jornada laboral es de lunes a viernes de 9:00 a 17:00 horas.
        Los viernes se puede salir una hora antes (16:00) durante los meses de verano.
        El trabajo remoto está permitido hasta 2 días por semana, previa coordinación con el supervisor.

        Beneficios: La empresa ofrece obra social, seguro de vida, y un bono anual equivalente
        al 15% del salario anual para empleados con más de 2 años de antigüedad.
        """
    },
    {
        "titulo": "Procedimientos de Seguridad Informática",
        "contenido": """
        Contraseñas: Todas las contraseñas deben tener al menos 12 caracteres, incluir mayúsculas,
        minúsculas, números y símbolos. Las contraseñas se deben cambiar cada 90 días.

        Acceso Remoto: Para conectarse remotamente, todos los empleados deben usar la VPN corporativa.
        El acceso se otorga solo con autorización del área de IT y requiere autenticación de dos factores.

        Backup de Datos: Todos los archivos importantes deben guardarse en el servidor corporativo.
        Los backups automáticos se realizan diariamente a las 2:00 AM.
        Los empleados no deben guardar información confidencial en dispositivos personales.

        Incidentes: Cualquier sospecha de violación de seguridad debe reportarse inmediatamente
        al área de IT llamando a la extensión 5555 o enviando un email a seguridad@empresa.com.
        """
    },
    {
        "titulo": "Manual de Gastos y Reintegros",
        "contenido": """
        Gastos de Viaje: La empresa reintegra hasta $50,000 por día en gastos de alojamiento
        y hasta $15,000 por día en comidas para viajes de trabajo.
        Los vuelos deben reservarse con al menos 15 días de anticipación para obtener autorización.

        Gastos de Capacitación: La empresa financia hasta $200,000 anuales por empleado
        en cursos y capacitaciones relacionadas con el trabajo.
        Los cursos deben ser aprobados previamente por Recursos Humanos.

        Reintegros Médicos: Los gastos médicos no cubiertos por la obra social son reintegrados
        hasta un máximo de $100,000 por año calendario.
        Se requiere presentar la factura original y el comprobante de la obra social.

        Proceso: Todos los reintegros deben solicitarse completando el formulario F-001
        y adjuntando los comprobantes correspondientes. El plazo máximo para solicitar reintegros es de 60 días.
        """
    },
    {
        "titulo": "Organigrama y Contactos Internos",
        "contenido": """
        Dirección General: María González (ext. 1001, mgonzalez@empresa.com)
        Gerente de Ventas: Carlos Rodríguez (ext. 2001, crodriguez@empresa.com)
        Gerente de IT: Ana López (ext. 3001, alopez@empresa.com)
        Recursos Humanos: Roberto Silva (ext. 4001, rsilva@empresa.com)

        Horarios de Atención:
        - Recursos Humanos: Lunes a Viernes 9:00 a 16:00
        - IT Support: Lunes a Viernes 8:00 a 18:00, Sábados 9:00 a 13:00
        - Administración: Lunes a Viernes 9:00 a 17:00

        Contactos de Emergencia:
        - Seguridad: ext. 5555
        - Mantenimiento: ext. 6666
        - Recepción: ext. 0
        """
    }
]

print(f"📄 Se crearon {len(documentos_empresa)} documentos empresariales de ejemplo:")
for i, doc in enumerate(documentos_empresa, 1):
    print(f"   {i}. {doc['titulo']}")

📄 Se crearon 4 documentos empresariales de ejemplo:
   1. Política de Recursos Humanos 2024
   2. Procedimientos de Seguridad Informática
   3. Manual de Gastos y Reintegros
   4. Organigrama y Contactos Internos


## Paso 1: Dividir los Documentos en Fragmentos

Los documentos largos necesitan dividirse en pedazos más pequeños para que el sistema pueda encontrar información específica. Es como hacer un índice detallado de un libro.

In [ ]:
# El "Text Splitter" es como un bibliotecario que divide documentos grandes
# en secciones manejables, manteniendo el contexto

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Cada fragmento tendrá máximo 500 caracteres
    chunk_overlap=50,      # 50 caracteres se superponen entre fragmentos para mantener contexto
    separators=["\n\n", "\n", ".", " "]  # Divide preferentemente por párrafos, luego oraciones
)

# Convertimos nuestros documentos al formato que entiende LangChain
documentos_langchain = []

for doc in documentos_empresa:
    # Cada documento se convierte en un objeto "Document" con contenido y metadata
    documento = Document(
        page_content=doc["contenido"],
        metadata={"titulo": doc["titulo"]}
    )
    documentos_langchain.append(documento)

# Dividimos todos los documentos en fragmentos más pequeños
fragmentos = text_splitter.split_documents(documentos_langchain)

print(f"📝 Documentos originales: {len(documentos_empresa)}")
print(f"🔪 Fragmentos creados: {len(fragmentos)}")
print(f"\n📋 Ejemplo de fragmento:")
print(f"Título: {fragmentos[0].metadata['titulo']}")
print(f"Contenido: {fragmentos[0].page_content[:200]}...")

📝 Documentos originales: 4
🔪 Fragmentos creados: 8

📋 Ejemplo de fragmento:
Título: Política de Recursos Humanos 2024
Contenido: Política de Vacaciones: Todos los empleados tienen derecho a 21 días hábiles de vacaciones anuales. 
        Las vacaciones deben solicitarse con al menos 15 días de anticipación. 
        Durante ene...


In [ ]:
print(f"Título: {fragmentos[2].metadata['titulo']}")
print(f"Contenido: {fragmentos[2].page_content[:200]}...")

Título: Procedimientos de Seguridad Informática
Contenido: Contraseñas: Todas las contraseñas deben tener al menos 12 caracteres, incluir mayúsculas, 
        minúsculas, números y símbolos. Las contraseñas se deben cambiar cada 90 días.
        
        Acce...


## Paso 2: Crear la Base de Conocimiento Vectorial

Ahora convertimos nuestros fragmentos de texto en "vectores" (listas de números) que el sistema puede comparar y buscar de forma inteligente.

In [ ]:
# Los "embeddings" convierten texto en vectores numéricos que representan el significado
# Usamos un modelo local multilenguaje que funciona excelente con español técnico
embeddings = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="intfloat/multilingual-e5-large"  # Modelo multilenguaje optimizado para español
)

print("Modelo de embeddings local configurado (multilingual-e5-large)")
print("Ventaja: No consume cuota de API, solo procesamiento local")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Modelo de embeddings local configurado (multilingual-e5-large)
Ventaja: No consume cuota de API, solo procesamiento local


In [ ]:
!pip install langchain_community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [ ]:
# ChromaDB será nuestra "biblioteca inteligente" donde guardamos los vectores
# Es como un bibliotecario que puede encontrar libros por su tema, no solo por título
from langchain_community.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")

vectorstore = Chroma.from_documents(
    documents=fragmentos,           # Los fragmentos de nuestros documentos
    embedding=embeddings,           # El modelo que convierte texto en vectores
    collection_name="documentos_empresa",  # Nombre de nuestra colección
    persist_directory="./chroma_db"  # Donde se guardan los datos (opcional)
)

print(f"Base de conocimiento vectorial creada con {len(fragmentos)} fragmentos")
print("El sistema ya puede buscar información por significado, no solo por palabras exactas")
print("Los embeddings se procesan localmente sin consumir cuota de Gemini")

Base de conocimiento vectorial creada con 8 fragmentos
El sistema ya puede buscar información por significado, no solo por palabras exactas
Los embeddings se procesan localmente sin consumir cuota de Gemini


In [ ]:
# Configuramos el modelo Gemini que generará las respuestas finales
# NOTA: Solo este componente consume cuota de API, los embeddings son locales
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",    # Modelo rápido y eficiente de Gemini
    temperature=0.1,             # Baja creatividad = respuestas más precisas y consistentes
    google_api_key=GOOGLE_API_KEY
)

print("Modelo Gemini configurado")
print("   Modelo: gemini-1.5-flash (rápido y preciso)")
print("   Temperatura: 0.1 (respuestas consistentes y factuales)")
print("   IMPORTANTE: Solo la generación final usa API de Gemini")

Modelo Gemini configurado
   Modelo: gemini-1.5-flash (rápido y preciso)
   Temperatura: 0.1 (respuestas consistentes y factuales)
   IMPORTANTE: Solo la generación final usa API de Gemini


## Paso 4: Crear el Template de Respuesta

Definimos cómo queremos que Gemini estructure sus respuestas. Es como darle instrucciones específicas sobre su rol y forma de responder.

In [ ]:
# El "prompt template" es como las instrucciones que le damos a un asistente
# Le decimos exactamente cómo debe comportarse y qué formato usar

template_respuesta = """
Sos un asistente experto en políticas y procedimientos empresariales.
Tu trabajo es responder preguntas basándote ÚNICAMENTE en la información
proporcionada en los documentos de la empresa.

INSTRUCCIONES IMPORTANTES:
1. Solo usa información que aparece explícitamente en los documentos
2. Si no encontrás la información específica, decilo claramente
3. Cita el documento o sección cuando sea posible
4. Sé preciso con números, fechas y procedimientos
5. Usa un tono profesional pero amigable

CONTEXTO DE LOS DOCUMENTOS:
{context}

PREGUNTA DEL EMPLEADO:
{question}

RESPUESTA:
"""

# Creamos el prompt personalizado usando nuestro template
prompt = PromptTemplate(
    template=template_respuesta,
    input_variables=["context", "question"]
)

print("Template de respuesta configurado")
print("El asistente seguirá instrucciones específicas para dar respuestas precisas")

Template de respuesta configurado
El asistente seguirá instrucciones específicas para dar respuestas precisas


## Paso 5: Ensamblar el Sistema RAG Completo

Ahora conectamos todas las piezas: la búsqueda vectorial, el modelo de lenguaje, y las instrucciones. ¡Es como armar un equipo de trabajo perfecto!

In [ ]:
# El "RetrievalQA" es el corazón de nuestro sistema RAG
# Conecta la búsqueda (Retrieval) con la generación (QA = Question Answering)

sistema_rag = RetrievalQA.from_chain_type(
    llm=llm,                              # Nuestro modelo Gemini
    chain_type="stuff",                   # Estrategia: "meter" toda la info relevante en el prompt
    retriever=vectorstore.as_retriever(   # Configuración del buscador
        search_kwargs={"k": 3}            # Buscar los 3 fragmentos más relevantes
    ),
    chain_type_kwargs={                   # Configuraciones adicionales
        "prompt": prompt,                 # Nuestras instrucciones personalizadas
        "verbose": False                  # No mostrar pasos internos (para mantenerlo limpio)
    },
    return_source_documents=True          # Devolver también los documentos fuente
)

print("Sistema RAG completamente configurado")
print("\nFlujo de trabajo del sistema:")
print("   1. Usuario hace una pregunta")
print("   2. El sistema busca los 3 fragmentos más relevantes (LOCAL)")
print("   3. Gemini lee esos fragmentos y genera una respuesta (API)")
print("   4. Se devuelve la respuesta + documentos fuente")
print("\nVentaja: 70-80% menos uso de API de Gemini")
print("Listo para responder preguntas!")

Sistema RAG completamente configurado

Flujo de trabajo del sistema:
   1. Usuario hace una pregunta
   2. El sistema busca los 3 fragmentos más relevantes (LOCAL)
   3. Gemini lee esos fragmentos y genera una respuesta (API)
   4. Se devuelve la respuesta + documentos fuente

Ventaja: 70-80% menos uso de API de Gemini
Listo para responder preguntas!


## Probando Nuestro Sistema RAG

Vamos a hacer varias preguntas para ver cómo nuestro sistema encuentra información específica y genera respuestas precisas.

In [ ]:
# Función auxiliar para mostrar respuestas de forma clara y educativa
def hacer_pregunta(pregunta, mostrar_fuentes=True):
    """
    Función que procesa una pregunta y muestra la respuesta de forma educativa

    Args:
        pregunta (str): La pregunta que queremos hacer al sistema
        mostrar_fuentes (bool): Si mostrar o no los documentos fuente
    """
    print(f"\n{'='*60}")
    print(f"PREGUNTA: {pregunta}")
    print(f"{'='*60}")

    # Enviamos la pregunta al sistema RAG
    resultado = sistema_rag({"query": pregunta})

    # Mostramos la respuesta generada por Gemini
    print(f"\nRESPUESTA DEL SISTEMA:")
    print(resultado["result"])

    # Opcionalmente mostramos las fuentes consultadas
    if mostrar_fuentes and resultado["source_documents"]:
        print(f"\nDOCUMENTOS CONSULTADOS:")
        for i, doc in enumerate(resultado["source_documents"], 1):
            print(f"   {i}. {doc.metadata['titulo']}")
            print(f"      Fragmento: {doc.page_content[:100]}...")

    return resultado

print("Función de prueba lista")
print("Ahora podemos hacer preguntas específicas sobre nuestros documentos empresariales")

Función de prueba lista
Ahora podemos hacer preguntas específicas sobre nuestros documentos empresariales


### Pregunta 1: Información sobre Vacaciones

In [ ]:
# Preguntamos sobre la política de vacaciones
resultado1 = hacer_pregunta("¿Cuántos días de vacaciones tengo al año y con cuánta anticipación debo solicitarlas?")


PREGUNTA: ¿Cuántos días de vacaciones tengo al año y con cuánta anticipación debo solicitarlas?

RESPUESTA DEL SISTEMA:
Tiene derecho a 21 días hábiles de vacaciones anuales.  Debe solicitarlas con al menos 15 días de anticipación. (Política de Vacaciones)

DOCUMENTOS CONSULTADOS:
   1. Política de Recursos Humanos 2024
      Fragmento: Política de Vacaciones: Todos los empleados tienen derecho a 21 días hábiles de vacaciones anuales. ...
   2. Política de Recursos Humanos 2024
      Fragmento: El trabajo remoto está permitido hasta 2 días por semana, previa coordinación con el supervisor.
   ...
   3. Manual de Gastos y Reintegros
      Fragmento: Gastos de Viaje: La empresa reintegra hasta $50,000 por día en gastos de alojamiento 
        y hast...


### Pregunta 2: Procedimientos de Seguridad

In [ ]:
# Preguntamos sobre requerimientos de contraseñas
resultado2 = hacer_pregunta("¿Cuáles son los requerimientos para las contraseñas de la empresa?")


PREGUNTA: ¿Cuáles son los requerimientos para las contraseñas de la empresa?

RESPUESTA DEL SISTEMA:
Las contraseñas de la empresa deben tener al menos 12 caracteres, incluyendo mayúsculas, minúsculas, números y símbolos.  Deben cambiarse cada 90 días.

DOCUMENTOS CONSULTADOS:
   1. Procedimientos de Seguridad Informática
      Fragmento: Contraseñas: Todas las contraseñas deben tener al menos 12 caracteres, incluir mayúsculas, 
        ...
   2. Procedimientos de Seguridad Informática
      Fragmento: Backup de Datos: Todos los archivos importantes deben guardarse en el servidor corporativo. 
       ...
   3. Política de Recursos Humanos 2024
      Fragmento: El trabajo remoto está permitido hasta 2 días por semana, previa coordinación con el supervisor.
   ...


### Pregunta 3: Reintegros de Gastos

In [ ]:
# Preguntamos sobre gastos de capacitación
resultado3 = hacer_pregunta("¿Cuánto dinero cubre la empresa para cursos de capacitación?")


PREGUNTA: ¿Cuánto dinero cubre la empresa para cursos de capacitación?

RESPUESTA DEL SISTEMA:
La empresa financia hasta $200.000 anuales por empleado en cursos y capacitaciones relacionadas con el trabajo.  Estos cursos deben ser aprobados previamente por Recursos Humanos.

DOCUMENTOS CONSULTADOS:
   1. Manual de Gastos y Reintegros
      Fragmento: Gastos de Viaje: La empresa reintegra hasta $50,000 por día en gastos de alojamiento 
        y hast...
   2. Política de Recursos Humanos 2024
      Fragmento: El trabajo remoto está permitido hasta 2 días por semana, previa coordinación con el supervisor.
   ...
   3. Organigrama y Contactos Internos
      Fragmento: Dirección General: María González (ext. 1001, mgonzalez@empresa.com)
        Gerente de Ventas: Carl...


### Pregunta 4: Información de Contactos

In [ ]:
# Preguntamos sobre contactos internos
resultado4 = hacer_pregunta("¿Cuál es la extensión para contactar al área de IT?")


PREGUNTA: ¿Cuál es la extensión para contactar al área de IT?

RESPUESTA DEL SISTEMA:
La extensión para contactar al área de IT es la 3001 (para Ana López, Gerente de IT) o la 5555 para reportar incidentes de seguridad.

DOCUMENTOS CONSULTADOS:
   1. Organigrama y Contactos Internos
      Fragmento: Dirección General: María González (ext. 1001, mgonzalez@empresa.com)
        Gerente de Ventas: Carl...
   2. Procedimientos de Seguridad Informática
      Fragmento: Backup de Datos: Todos los archivos importantes deben guardarse en el servidor corporativo. 
       ...
   3. Procedimientos de Seguridad Informática
      Fragmento: Contraseñas: Todas las contraseñas deben tener al menos 12 caracteres, incluir mayúsculas, 
        ...


### Pregunta 5: Pregunta que NO está en los documentos

In [ ]:
# Probamos qué pasa cuando preguntamos algo que no está en nuestros documentos
resultado5 = hacer_pregunta("¿Cuál es el procedimiento para solicitar una computadora nueva?")


PREGUNTA: ¿Cuál es el procedimiento para solicitar una computadora nueva?

RESPUESTA DEL SISTEMA:
Lo siento, pero la información proporcionada no incluye un procedimiento para solicitar una computadora nueva.  Los documentos describen políticas de contraseñas, acceso remoto, reintegros médicos y datos de contacto del personal, pero no hay información sobre la solicitud de equipos.

DOCUMENTOS CONSULTADOS:
   1. Procedimientos de Seguridad Informática
      Fragmento: Contraseñas: Todas las contraseñas deben tener al menos 12 caracteres, incluir mayúsculas, 
        ...
   2. Manual de Gastos y Reintegros
      Fragmento: Reintegros Médicos: Los gastos médicos no cubiertos por la obra social son reintegrados 
        has...
   3. Organigrama y Contactos Internos
      Fragmento: Dirección General: María González (ext. 1001, mgonzalez@empresa.com)
        Gerente de Ventas: Carl...


## Análisis del Sistema: ¿Cómo funciona internamente?

Vamos a "levantar el capó" y ver exactamente qué está pasando cuando hacemos una pregunta.

In [ ]:
# Función para mostrar el proceso interno paso a paso
def analizar_proceso(pregunta):
    """
    Función educativa que muestra todos los pasos internos del proceso RAG
    """
    print(f"\nANALISIS PASO A PASO")
    print(f"Pregunta: {pregunta}")
    print("\n" + "="*50)

    # PASO 1: Búsqueda de documentos relevantes
    print("PASO 1: Búsqueda Vectorial (LOCAL - sin usar API)")
    documentos_relevantes = vectorstore.similarity_search(pregunta, k=3)

    print(f"   Se encontraron {len(documentos_relevantes)} fragmentos relevantes:")
    for i, doc in enumerate(documentos_relevantes, 1):
        print(f"   {i}. De: {doc.metadata['titulo']}")
        print(f"      Contenido: {doc.page_content[:150]}...\n")

    # PASO 2: Construcción del contexto
    print("PASO 2: Construcción del Contexto (LOCAL)")
    contexto = "\n\n".join([doc.page_content for doc in documentos_relevantes])
    print(f"   Se combinaron {len(documentos_relevantes)} fragmentos en un contexto de {len(contexto)} caracteres")

    # PASO 3: Generación del prompt final
    print("\nPASO 3: Prompt Final para Gemini (LOCAL)")
    prompt_final = prompt.format(context=contexto, question=pregunta)
    print("   Longitud del prompt:", len(prompt_final), "caracteres")
    print("   Primeras líneas del prompt:")
    print("   " + "\n   ".join(prompt_final.split("\n")[:8]))

    # PASO 4: Respuesta del modelo
    print("\nPASO 4: Generación de Respuesta (USA API DE GEMINI)")
    resultado = sistema_rag({"query": pregunta})
    print(f"   Respuesta generada: {len(resultado['result'])} caracteres")
    print(f"   Respuesta: {resultado['result']}")

    return resultado

# Analicemos una pregunta específica
analisis = analizar_proceso("¿Cuándo se puede trabajar remotamente?")

## Experimentando con Diferentes Tipos de Búsqueda

Vamos a comparar diferentes estrategias de búsqueda para entender mejor cómo funciona la búsqueda semántica.

In [ ]:
# Función para comparar búsqueda semántica vs búsqueda por palabras clave
def comparar_busquedas(pregunta, palabras_clave):
    """
    Compara búsqueda semántica con búsqueda tradicional por palabras
    """
    print(f"\n🔄 COMPARACIÓN DE BÚSQUEDAS")
    print(f"Pregunta: {pregunta}")
    print(f"Palabras clave: {palabras_clave}")
    print("\n" + "="*60)

    # Búsqueda semántica (por significado)
    print("🧠 BÚSQUEDA SEMÁNTICA (por significado):")
    docs_semanticos = vectorstore.similarity_search(pregunta, k=3)
    for i, doc in enumerate(docs_semanticos, 1):
        print(f"   {i}. {doc.metadata['titulo']}: {doc.page_content[:100]}...")

    # Búsqueda tradicional (por palabras exactas)
    print(f"\n🔤 BÚSQUEDA TRADICIONAL (palabras exactas: '{palabras_clave}'):")
    # Simulamos búsqueda tradicional buscando la palabra en cada documento
    docs_tradicional = []
    for doc in fragmentos:
        if palabras_clave.lower() in doc.page_content.lower():
            docs_tradicional.append(doc)

    if docs_tradicional:
        for i, doc in enumerate(docs_tradicional[:3], 1):
            print(f"   {i}. {doc.metadata['titulo']}: {doc.page_content[:100]}...")
    else:
        print("   ❌ No se encontraron coincidencias exactas")

    print(f"\n📊 RESULTADOS:")
    print(f"   Búsqueda semántica: {len(docs_semanticos)} resultados")
    print(f"   Búsqueda tradicional: {len(docs_tradicional)} resultados")

# Probemos con ejemplos que muestren la diferencia
comparar_busquedas(
    "¿Puedo trabajar desde mi casa?",
    "casa"  # La palabra "casa" no aparece en los docs, pero "remoto" sí
)

comparar_busquedas(
    "¿Qué pasa si alguien hackea el sistema?",
    "hack"  # "hack" no está, pero "incidente de seguridad" sí
)

## Mejorando el Sistema: Configuraciones Avanzadas

Vamos a experimentar con diferentes configuraciones para optimizar nuestro sistema RAG.

In [ ]:
# Creamos una versión más avanzada del sistema con mejores configuraciones

# 1. Configuración de búsqueda más inteligente
retriever_avanzado = vectorstore.as_retriever(
    search_type="mmr",          # Maximum Marginal Relevance: evita resultados repetitivos
    search_kwargs={
        "k": 4,                 # Buscar 4 fragmentos
        "fetch_k": 10,          # Considerar 10 candidatos antes de elegir los mejores 4
        "lambda_mult": 0.7      # Balance entre relevancia y diversidad
    }
)

# 2. Template mejorado con más estructura
template_mejorado = """
Sos un asistente especializado en recursos humanos y políticas empresariales.
Tu misión es proporcionar información precisa y útil basada únicamente en
los documentos oficiales de la empresa.

FORMATO DE RESPUESTA:
- Comenzá con un resumen directo
- Proporcioná detalles específicos (números, fechas, procedimientos)
- Si hay pasos a seguir, numeralos claramente
- Mencioná qué documento contiene la información
- Si algo no está claro en los documentos, decilo explícitamente

DOCUMENTOS DISPONIBLES:
{context}

CONSULTA DEL EMPLEADO:
{question}

RESPUESTA ESTRUCTURADA:
"""

prompt_mejorado = PromptTemplate(
    template=template_mejorado,
    input_variables=["context", "question"]
)

# 3. Sistema RAG mejorado
sistema_rag_mejorado = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever_avanzado,
    chain_type_kwargs={"prompt": prompt_mejorado},
    return_source_documents=True
)

print("⚡ Sistema RAG mejorado configurado")
print("\nMejoras implementadas:")
print("✅ Búsqueda MMR (menos resultados repetitivos)")
print("✅ Análisis de más candidatos (10 vs 3)")
print("✅ Template de respuesta más estructurado")
print("✅ Balance optimizado entre relevancia y diversidad")

In [ ]:
# Comparemos el sistema original con el mejorado
def comparar_sistemas(pregunta):
    print(f"\n⚔️ COMPARACIÓN DE SISTEMAS")
    print(f"Pregunta: {pregunta}")
    print("\n" + "="*70)

    print("🔵 SISTEMA ORIGINAL:")
    resultado_original = sistema_rag({"query": pregunta})
    print(resultado_original["result"])

    print("\n🟢 SISTEMA MEJORADO:")
    resultado_mejorado = sistema_rag_mejorado({"query": pregunta})
    print(resultado_mejorado["result"])

    print(f"\n📊 ANÁLISIS:")
    print(f"   Original: {len(resultado_original['source_documents'])} documentos, {len(resultado_original['result'])} caracteres")
    print(f"   Mejorado: {len(resultado_mejorado['source_documents'])} documentos, {len(resultado_mejorado['result'])} caracteres")

# Probemos con una pregunta compleja
comparar_sistemas("Quiero tomar vacaciones en enero y también necesito hacer un curso. ¿Cuáles son todos los pasos y limitaciones?")

## Ejercicio Práctico: Agregando Nuevos Documentos

Vamos a simular cómo agregar nuevos documentos a nuestro sistema RAG en funcionamiento.

In [ ]:
# Simulamos que llegan nuevos documentos empresariales
nuevos_documentos = [
    {
        "titulo": "Protocolo de Trabajo Híbrido 2024",
        "contenido": """
        Modalidades de Trabajo: A partir del 1 de marzo de 2024, todos los empleados pueden optar por:
        1. Trabajo presencial 100% (oficina todos los días)
        2. Trabajo híbrido (3 días oficina, 2 días remoto)
        3. Trabajo remoto total (solo para puestos específicos aprobados por gerencia)

        Días Obligatorios en Oficina: Todos los martes son obligatorios para reuniones de equipo.
        Los jueves también son recomendados para colaboración presencial.

        Equipamiento: Los empleados remotos pueden solicitar silla ergonómica y monitor adicional.
        El reintegro por internet domiciliario es de hasta $8,000 mensuales.

        Evaluación: El rendimiento se mide por objetivos cumplidos, no por horas trabajadas.
        Las evaluaciones se realizan trimestralmente con métricas específicas por área.
        """
    },
    {
        "titulo": "Programa de Bienestar Laboral",
        "contenido": """
        Gimnasio: La empresa tiene convenio con FitClub. Reintegro del 50% de la cuota mensual,
        hasta un máximo de $15,000 por mes por empleado.

        Salud Mental: Cobertura de 8 sesiones anuales de psicología con profesionales del convenio.
        También disponible programa de mindfulness los miércoles de 12:00 a 13:00.

        Flexibilidad Horaria: Se puede ingresar entre 8:00 y 10:00 AM, completando las 8 horas diarias.
        Los viernes se puede salir a las 15:00 si se completaron las horas semanales.

        Días de Bienestar: 2 días adicionales por año para actividades de bienestar personal,
        no acumulables y deben usarse en el año calendario.
        """
    }
]

print(f"📥 Llegaron {len(nuevos_documentos)} documentos nuevos:")
for doc in nuevos_documentos:
    print(f"   • {doc['titulo']}")

In [ ]:
# Función para agregar nuevos documentos al sistema existente
def agregar_documentos(nuevos_docs, vectorstore):
    """
    Agrega nuevos documentos a nuestro sistema RAG existente
    """
    print("🔄 Procesando nuevos documentos...")

    # Convertir a formato LangChain
    docs_langchain = []
    for doc in nuevos_docs:
        documento = Document(
            page_content=doc["contenido"],
            metadata={"titulo": doc["titulo"]}
        )
        docs_langchain.append(documento)

    # Dividir en fragmentos
    nuevos_fragmentos = text_splitter.split_documents(docs_langchain)
    print(f"📝 Se crearon {len(nuevos_fragmentos)} nuevos fragmentos")

    # Agregar al vectorstore existente
    vectorstore.add_documents(nuevos_fragmentos)
    print(f"✅ Documentos agregados exitosamente a la base de conocimiento")

    return len(nuevos_fragmentos)

# Agregamos los nuevos documentos
fragmentos_nuevos = agregar_documentos(nuevos_documentos, vectorstore)

print(f"\n📊 Estado actual del sistema:")
print(f"   Documentos originales: {len(documentos_empresa)} docs")
print(f"   Documentos nuevos: {len(nuevos_documentos)} docs")
print(f"   Total de fragmentos en la base: {len(fragmentos) + fragmentos_nuevos} fragmentos")

In [ ]:
# Probemos preguntas sobre los nuevos documentos
print("🆕 Probando preguntas sobre los documentos recién agregados:\n")

hacer_pregunta("¿Qué días son obligatorios ir a la oficina?")
hacer_pregunta("¿La empresa cubre gastos de gimnasio?")
hacer_pregunta("¿Cuántos días de bienestar tengo al año?")

## Casos de Uso Avanzados

Exploremos escenarios más complejos que muestran el verdadero poder de RAG en entornos empresariales.

In [ ]:
# Función para manejar consultas complejas que requieren múltiples documentos
def consulta_compleja(pregunta):
    """
    Maneja consultas que pueden necesitar información de múltiples documentos
    """
    print(f"\n🧩 CONSULTA COMPLEJA")
    print(f"Pregunta: {pregunta}")
    print("\n" + "="*60)

    # Configuramos para buscar más documentos
    retriever_complejo = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 6, "fetch_k": 15}  # Más documentos para consultas complejas
    )

    # Sistema temporal con más contexto
    sistema_complejo = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever_complejo,
        chain_type_kwargs={"prompt": prompt_mejorado},
        return_source_documents=True
    )

    resultado = sistema_complejo({"query": pregunta})

    print(f"🤖 RESPUESTA INTEGRAL:")
    print(resultado["result"])

    print(f"\n📚 DOCUMENTOS CONSULTADOS ({len(resultado['source_documents'])}):")
    documentos_unicos = list(set([doc.metadata['titulo'] for doc in resultado['source_documents']]))
    for i, titulo in enumerate(documentos_unicos, 1):
        print(f"   {i}. {titulo}")

    return resultado

# Probemos con preguntas que necesitan múltiples fuentes
consulta_compleja("Soy nuevo empleado, trabajaré híbrido, y quiero saber todo sobre mis beneficios, vacaciones, y qué documentos necesito conocer")

consulta_compleja("Quiero maximizar mis beneficios este año: vacaciones, capacitación, bienestar y trabajo remoto. ¿Cómo lo planifico?")

---

# Glosario de Términos

**RAG (Retrieval Augmented Generation)**: Sistema que combina búsqueda de información con generación de texto. Primero busca documentos relevantes, luego genera respuestas basadas en esa información específica.

**Embeddings**: Representaciones numéricas (vectores) de texto que capturan el significado semántico. Permiten comparar textos por significado, no solo por palabras exactas.

**Vector Store/Base Vectorial**: Base de datos especializada en almacenar y buscar vectores. Permite encontrar información similar por significado matemático.

**ChromaDB**: Base de datos vectorial gratuita y fácil de usar, ideal para proyectos educativos y prototipos RAG.

**LangChain**: Librería de Python que simplifica la construcción de aplicaciones con modelos de lenguaje, especialmente sistemas RAG.

**Text Splitter**: Herramienta que divide documentos largos en fragmentos más pequeños, manteniendo el contexto entre fragmentos.

**Chunk/Fragmento**: Pedazo pequeño de texto (típicamente 200-1000 caracteres) en que se dividen los documentos para procesar.

**Retriever/Recuperador**: Componente que busca y devuelve los fragmentos más relevantes de la base vectorial para una pregunta específica.

**Prompt Template**: Plantilla que define cómo estructurar las instrucciones para el modelo de IA, incluyendo contexto y pregunta.

**Similarity Search**: Búsqueda por similitud semántica, encuentra textos con significado parecido aunque usen palabras diferentes.

**MMR (Maximum Marginal Relevance)**: Estrategia de búsqueda que balancea relevancia con diversidad, evitando resultados muy similares entre sí.

**Temperature**: Parámetro que controla la creatividad del modelo. Valores bajos (0.0-0.3) dan respuestas más precisas y consistentes.

**Context Window**: Cantidad máxima de texto que un modelo puede procesar de una vez (contexto + pregunta + respuesta).

**Gemini**: Familia de modelos de IA de Google, incluyendo capacidades de texto, código, y comprensión multimodal.

---

# Guía de Estudio: Preguntas y Respuestas

## Conceptos Fundamentales

**P1: ¿Cuál es la diferencia principal entre un modelo de IA tradicional y un sistema RAG?**

R: Un modelo tradicional solo puede responder con información de su entrenamiento (datos hasta una fecha específica). RAG permite que el modelo acceda a documentos actualizados y específicos de tu empresa, combinando búsqueda de información con generación de respuestas.

**P2: ¿Por qué necesitamos dividir documentos en fragmentos pequeños?**

R: Los modelos tienen límites en la cantidad de texto que pueden procesar de una vez. Fragmentos pequeños permiten encontrar información específica y relevante, en lugar de enviar documentos completos que pueden contener información irrelevante.

**P3: ¿Qué ventajas tiene la búsqueda semántica sobre la búsqueda por palabras clave?**

R: La búsqueda semántica entiende el significado, no solo palabras exactas. Por ejemplo, puede conectar "trabajar desde casa" con "trabajo remoto" aunque usen palabras diferentes.

## Aspectos Técnicos

**P4: ¿Qué hace exactamente el parámetro 'k' en la configuración del retriever?**

R: Define cuántos fragmentos de documentos se devuelven como contexto. k=3 significa que se buscarán los 3 fragmentos más relevantes para responder la pregunta.

**P5: ¿Por qué usamos temperature=0.1 en lugar de un valor más alto?**

R: Para obtener respuestas consistentes y factuales. Valores altos aumentan la creatividad pero pueden generar información incorrecta. En contextos empresariales, la precisión es más importante que la creatividad.

**P6: ¿Cuál es la función del chunk_overlap en el text splitter?**

R: Mantiene continuidad entre fragmentos. Si una información importante está en el límite entre dos fragmentos, el overlap asegura que no se pierda el contexto.

## Implementación Práctica

**P7: ¿Cómo agregarías un nuevo documento al sistema sin reiniciarlo?**

R: Usando vectorstore.add_documents(nuevos_fragmentos). El sistema puede actualizar la base de conocimiento dinámicamente sin perder documentos existentes.

**P8: ¿Qué limitaciones tiene el enfoque "stuff" para combinar documentos?**

R: Está limitado por el context window del modelo. Si se necesita consultar muchos documentos largos, se podría exceder el límite. Alternativas incluyen "map-reduce" o "refine".

**P9: ¿Cómo optimizarías el sistema para documentos muy largos?**

R: Usar chunks más pequeños con mayor overlap, implementar búsqueda jerárquica, o usar estrategias como "map-reduce" que procesan documentos en múltiples pasos.

## Casos de Uso

**P10: ¿En qué situaciones RAG es mejor que fine-tuning de un modelo?**

R: Cuando la información cambia frecuentemente, cuando necesitas transparencia sobre las fuentes, cuando tienes documentos específicos de dominio, o cuando necesitas actualizar conocimiento sin reentrenar modelos.

**P11: ¿Qué tipos de documentos empresariales se benefician más de RAG?**

R: Políticas de RR.HH., manuales de procedimientos, documentación técnica, bases de conocimiento de soporte, regulaciones legales, y cualquier información que se actualice regularmente.

**P12: ¿Cómo evaluarías si tu sistema RAG está funcionando bien?**

R: Verificando que las respuestas citen fuentes correctas, que no invente información, que encuentre documentos relevantes, y que las respuestas sean precisas y útiles para los usuarios finales.

## Ejercicios Prácticos Sugeridos

1. **Modifica el prompt template** para que las respuestas incluyan siempre el nombre del documento fuente.

2. **Experimenta con diferentes valores de k** (1, 3, 5, 10) y observa cómo cambian las respuestas.

3. **Agrega un sistema de puntuación** que califique qué tan seguro está el sistema de cada respuesta.

4. **Implementa filtros por tipo de documento** para que los usuarios puedan buscar solo en categorías específicas.

5. **Crea un log de preguntas frecuentes** para identificar qué información falta en tus documentos.

---

## Próximos Pasos Recomendados

- **Integración con PDFs**: Usar librerías como PyPDF2 o pdfplumber para cargar documentos reales
- **Interfaz web**: Crear una aplicación con Streamlit o FastAPI para uso empresarial
- **Múltiples formatos**: Agregar soporte para Word, Excel, y otros formatos empresariales
- **Seguridad**: Implementar autenticación y control de acceso por documento
- **Métricas**: Agregar logging y analytics para mejorar continuamente el sistema